In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

input_file_name = 'BL -returndata.xlsx'

In [2]:
headers = '{}\n---------------\n\n'
contents = '{}:\n{}\n\n'


class MarketData:
    def __init__(self):
        self.df = None
        self.covariances = None  # n * n
        pass

    def update_data(self, path: str, sheet_name: str, usecols: list):
        self.df = pd.read_excel(path, sheet_name=sheet_name, usecols=usecols)
        self.covariances = self.df.cov()

    def __str__(self):
        return contents.format('Covariance matrix', self.covariances)


class CAPM:
    def __init__(self, market_data: MarketData, market_weights: np.ndarray, aversion: float, tau: float):
        self.market_data = market_data
        self.market_weights = market_weights  # n
        self.aversion = aversion
        self.tau = tau

    def market_variance(self) -> float:
        return float(self.market_weights @ self.market_data.covariances @ self.market_weights)

    def market_std_dev(self) -> float:
        return np.sqrt(self.market_variance())

    def market_expected_excess_return(self) -> float:
        return self.aversion * self.market_variance()

    def market_sharpe_ration(self) -> float:
        return self.market_expected_excess_return() / self.market_std_dev()

    def asset_expected_excess_returns(self) -> np.ndarray:  # n
        return self.aversion * np.dot(self.market_data.covariances, self.market_weights)

    def expected_return_distribution(self) -> np.ndarray:  # n * n
        return self.tau * self.market_data.covariances

    def __str__(self):
        result = contents.format('Market weights', self.market_weights)
        result += contents.format('Aversion', self.aversion)
        result += contents.format('Tau', self.tau)
        result += contents.format('Market variance', self.market_variance())
        result += contents.format('Market standard deviation', self.market_std_dev())
        result += contents.format('Market expected access return', self.market_expected_excess_return())
        result += contents.format('Market Sharpe ratio', self.market_sharpe_ration())
        result += contents.format('Asset expected excess returns', self.asset_expected_excess_returns())
        result += contents.format('Expected return distribution (tau sigma)', self.expected_return_distribution())
        return result


class Views:
    def __init__(self, market_data: MarketData, views: np.ndarray, view_excess_returns: np.ndarray, tau_v: float):
        self.market_data = market_data
        self.views = views  # k * n
        self.view_excess_returns = view_excess_returns  # k
        self.tau_v = tau_v

    def uncertainty(self) -> np.ndarray:  # k * k
        return self.tau_v * (self.views @ self.market_data.covariances @ np.transpose(self.views))

    def __str__(self):
        result = contents.format('Views', self.views)
        result += contents.format('View excess returns', self.view_excess_returns)
        result += contents.format('Tau_v', self.tau_v)
        result += contents.format('Uncertainty (Omega)', self.uncertainty())
        return result


class BlackLitterman:
    def __init__(self, market_data: MarketData, capm: CAPM, views: Views):
        self.market_data = market_data
        self.capm = capm
        self.views = views

    def __helper(self):  # n * k
        return (self.capm.expected_return_distribution() @ np.transpose(self.views.views) @
                np.linalg.inv(
                    self.views.views @ self.capm.expected_return_distribution() @ np.transpose(self.views.views) +
                    self.views.uncertainty()))

    def posterior_returns(self) -> np.ndarray:  # n
        return (self.capm.asset_expected_excess_returns() + self.__helper() @
                (self.views.view_excess_returns - self.views.views @ self.capm.asset_expected_excess_returns())
                )

    def posterior_return_distribution(self) -> np.ndarray:  # n * n
        return (self.market_data.covariances + self.capm.expected_return_distribution() -
                np.dot(self.__helper() @ self.views.views, self.capm.expected_return_distribution())
                )

    def optimal_weights(self):
        initial_weights = np.ones(len(self.market_data.covariances))

        def sharpe(weights):
            return -np.dot(self.posterior_returns(), weights) / np.sqrt(
                weights @ self.posterior_return_distribution() @ weights
            )

        constraints = ({'type': 'eq', 'fun': lambda weights: weights.sum() - 1.0})
        result = minimize(sharpe, initial_weights, method='SLSQP', constraints=constraints)
        return result.x

    def expected_excess_return(self) -> float:
        weights = self.optimal_weights()
        return np.dot(weights, self.posterior_returns())

    def variance(self) -> float:
        weights = self.optimal_weights()
        return float(weights @ self.posterior_return_distribution() @ weights)

    def std_dev(self) -> float:
        return np.sqrt(self.variance())

    def sharpe_ration(self) -> float:
        return self.expected_excess_return() / self.std_dev()

    def __str__(self):
        prior = headers.format('Prior')
        prior += self.market_data.__str__()
        prior += self.capm.__str__()
        view = headers.format('Views')
        view += self.views.__str__()
        posterior = headers.format('Posterior')
        posterior += contents.format('Posterior returns', self.posterior_returns())
        posterior += contents.format('Posterior return distribution', self.posterior_return_distribution())
        optimization = headers.format('Optimization')
        optimization += contents.format('Optimal weights', self.optimal_weights())
        optimization += contents.format('Expected access return', self.expected_excess_return())
        optimization += contents.format('Variance', self.variance())
        optimization += contents.format('Standard deviation', self.std_dev())
        optimization += contents.format('Sharpe ratio', self.sharpe_ration())
        return prior + view + posterior + optimization

In [3]:
parameter_sets = [[[0.5, 0.4, 0.1], 3.0, 0.1, [[1.0, 0.0, 0.0], [0.0, 1.0, -1.0]], [0.015, 0.03], 0.1],
                  [[0.5, 0.4, 0.1], 3.0, 0.1, [[1.0, 0.0, 0.0], [0.0, 1.0, -1.0]], [0.015, 0.03], 0.01],
                  [[0.5, 0.4, 0.1], 3.0, 0.1, [[1.0, -1.0, 0.0], [0.0, 0.0, 1.0]], [0.02, 0.015], 0.1]
                  ]

market_data = MarketData()
market_data.update_data(input_file_name, 'Excess Returns', ['US Equity', 'Foreign EQ', 'Emerging EQ'])
for param_set in parameter_sets:
    capm = CAPM(market_data, param_set[0], param_set[1], param_set[2])
    views = Views(market_data, param_set[3], param_set[4], param_set[5])
    black_litterman = BlackLitterman(market_data, capm, views)
    print(black_litterman)
    print('\n\n\n')

Prior
---------------

Covariance matrix:
             US Equity  Foreign EQ  Emerging EQ
US Equity     0.001846    0.001599     0.002031
Foreign EQ    0.001599    0.002438     0.002393
Emerging EQ   0.002031    0.002393     0.004383

Market weights:
[0.5, 0.4, 0.1]

Aversion:
3.0

Tau:
0.1

Market variance:
0.0019293205298343199

Market standard deviation:
0.043924031347706685

Market expected access return:
0.00578796158950296

Market Sharpe ratio:
0.13177209404312007

Asset expected excess returns:
[0.00529611 0.00604146 0.00723321]

Expected return distribution (tau sigma):
             US Equity  Foreign EQ  Emerging EQ
US Equity     0.000185    0.000160     0.000203
Foreign EQ    0.000160    0.000244     0.000239
Emerging EQ   0.000203    0.000239     0.000438

Views
---------------

Views:
[[1.0, 0.0, 0.0], [0.0, 1.0, -1.0]]

View excess returns:
[0.015, 0.03]

Tau_v:
0.1

Uncertainty (Omega):
          0         1
0  0.000185 -0.000043
1 -0.000043  0.000204

Posterior
---------